# This notebook will be mainly used for the capstone project.

## Importing necessary library
I need: pandas, Numpy, requests, Beautifulsoup library

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
print('Importing necessary lib: Done!')

Importing necessary lib: Done!


### Defining the url from wikipedia
The url is: https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969 which contains a list of post code with borough and neighbourhood

In [5]:
## Search Toronto neighborhoods 
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
html=requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

### Find table object

In [27]:
table = soup.findAll('table')
trs = table[0].findAll('tr')

# helper method to extracting all information from a list of 'tr' tag inside table
def find_all_element(trs):
    result = []
    for tr in trs:
        td = tr.findAll('td')
        # if tr contain td tags other than 3, continue
        if len(td) !=3:
            continue
        postcode = td[0].text.replace('\n','')
        borough = td[1].text.replace('\n','')
        
        # only processing data other than 'Not assigned'
        if borough == 'Not assigned':
            continue
        neigh = td[2].text.replace('\n','')
        if neigh == 'Not assigned':
            neigh = borough
        buffer_dict = {'POST_CODE':postcode,'BOROUGH':borough, 'neighbour':neigh.strip()}
        result.append(buffer_dict)
    return pd.DataFrame(result)  

df = find_all_element(trs)

In [28]:
df.head(20)

,POST_CODE,BOROUGH,neighbour
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [29]:
df.shape

(103, 3)